In [2]:
%run notebook-import.py
from python_implementations import Stack
from python_implementations import Queue

# Chapter 10.1: Stacks and queues

## Problem 1

Using Figure 10.1 as a model, illustrate the result of each operation in the sequence PUSH.S(4), PUSH.S(1), PUSH.S(3), POP.S(), PUSH.S(8), and POP.S() on an initially empty stack S stored in array S[1..6].

In [3]:
S = Stack(6)
S.push(4)
print(S)
S.push(1)
print(S)
S.push(3)
print(S)
S.pop()
print(S)
S.push(8)
print(S)
S.pop()
print(S)

[4]
[4, 1]
[4, 1, 3]
[4, 1]
[4, 1, 8]
[4, 1]


## Problem 2

Explain how to implement two stacks in one array $A[1::n]$ in such a way that neither stack overflows unless the total number of elements in both stacks together is n. The PUSH and POP operations should run in O(1).

### My answer

The first stack can be implemented so that "top" starts from the first index.
- push would increment "top" by 1.
- pop would decrement "top" by 1.

The second stack can be implemented so that "top" starts from the last index.
- push would decrement "top" by 1.
- pop would increment "top" by 1

## Problem 3

Using Figure 10.2 as a model, illustrate the result of each operation in the sequence ENQUEUE.Q(4), ENQUEUE.Q(1), ENQUEUE.Q(3), DEQUEUE.Q(), ENQUEUE.Q(8), and DEQUEUE.Q() on an initially empty queue Q stored in array $Q[1..6]$.

In [4]:
Q = Queue(6)
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")

Model: [None, None, None, None, None, None, None].
Simpler representation: [].


In [5]:
Q = Queue(6)
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")
Q.enqueue(4)
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")
Q.enqueue(3)
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")
Q.dequeue()
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")
Q.enqueue(8)
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")
Q.dequeue()
print(f"Model: {Q.queue}.")
print(f"Simpler representation: {Q}.")

Model: [None, None, None, None, None, None, None].
Simpler representation: [].
Model: [4, None, None, None, None, None, None].
Simpler representation: [4].
Model: [4, 3, None, None, None, None, None].
Simpler representation: [4, 3].
Model: [4, 3, None, None, None, None, None].
Simpler representation: [3].
Model: [4, 3, 8, None, None, None, None].
Simpler representation: [3, 8].
Model: [4, 3, 8, None, None, None, None].
Simpler representation: [8].


## Problem 4
Rewrite ENQUEUE and DEQUEUE to detect underflow and overflow of a queue.

### My Answer

This was done in my python implementation of enque and dequeue, but I can do this in pseudo-code as well.

```
// n is the size of the array.
// array index starts from 1 per Cormen's convention

    ENQUEUE(Q,x):
    if (Q.head - Q.tail) % n == 1:
        raise Exception("Overflow")
    else:
        Q[Q.tail] = x
        if Q.tail == n:
            Q.tail = 1
        else:
            Q.tail++
```

```
// n is the size of the array.
// array index starts from 1 per Cormen's convention

    DEQUEUE(Q):
    if Q.head == Q.tail:
        raise Exception("Underflow")
    else:
        x = Q[Q.head]
        if Q.head == n:
            Q.head = 1
        else:
            Q.head++
        return x
```

## Problem 5

Whereas a stack allows insertion and deletion of elements at only one end, and a queue allows insertion at one end and deletion at the other end, a deque (double- ended queue) allows insertion and deletion at both ends. Write four $O(1)$-time procedures to insert elements into and delete elements from both ends of a deque implemented by an array.


### My solution

In python_implementations, I created a subclass of Queue with 2 additional methods, which I named "front_enqueue" and "back_enqueu"

## Problem 6

Show how to implement a queue using two stacks. Analyze the running time of the queue operations.

### My solution

- There will be a "enque" stack and a "denque" stack.
- A variable that keeps track of the "state" of the queue.
- The state of the queue depends on the last operation that was performed on the queue:
    - 0 if the last operation was enque
    - 1 if the last operation was deque
    - The state variable will be initialized with 0
- When an element x is enqued:
    - if state == 0, then x is pushed into the enque_stack.
    - if state == 1, then each element of the denque_stack is popped then pushed into the enque_stack, then x is pushed into the enque_stack. Then, state gets changed to 0.
- When dequeue is used:
    - if state == 1, then x is popped from the denque_stack.
    - if state == 0, then each element of the enque_stack is popped then pushed into the denque_stack, then denque_stack is popped. Then, state gets changed to 1.

Complexity:
- O(1) the same operation is repeated (best case)
- O(n) if the operations alternate (worst case)

Note: 
- It is also possible to set it up so that one operation is always O(1), and the other one is O(2n):
    - copy into the extra stack, perform either push or poop, then copy back to the other

In [120]:
class Queue_stack:
    
    def __init__(self, max_cap):
        self.head = 0
        self.tail = 0
        self.state = 0
        self.max_cap = max_cap
        self.enque_stack = Stack(self.max_cap)
        self.denque_stack = Stack(self.max_cap)
    
    def is_full(self):
        return self.head == (self.tail + 1)%(self.max_cap + 1)

    def is_empty(self):
        return self.tail == self.head 
    
    def overflow(self):
        raise Exception("Overflow: The queue is full. No elements can be enqueued into the queue.")

    def underflow(self):
        raise Exception("Underflow: The queue is empty. No elements can be dequeued from the queue.")
    
    def move_data(self, source, target):
        while not source.is_empty():
            target.push(source.pop())
    
    def enqueue(self,x):
        if self.is_full():
            self.overflow()
        else:
            self.tail = (self.tail + 1)%(self.max_cap + 1)
            if self.state == 1:
                self.move_data(self.denque_stack, self.enque_stack)
                self.state = 0
            self.enque_stack.push(x)
    
    def denqueue(self):
        if self.is_empty():
            self.underflow()
        else:
            self.head = (self.head + 1)%(self.max_cap + 1)
            if self.state == 0:
                self.move_data(self.enque_stack, self.denque_stack)
                self.state = 1
            return self.denque_stack.pop()
    
    def __str__(self):
        if self.state == 0:
            return str(self.enque_stack)
        else:
            return "["+",".join(str(self.denque_stack)[1:-1].split(",")[::-1])+"]"
            

In [121]:
test = Queue_stack(10)

In [122]:
test = Queue_stack(10)
print(test.enque_stack)
print(test.denque_stack)

[]
[]


In [123]:
for i in range(10):
    if i < 5:
        test.enqueue(i)
    else:
        test.denqueue()
    print(test)

        

[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[ 1, 2, 3,4]
[ 2, 3,4]
[ 3,4]
[4]
[]


## Problem 7

Show how to implement a stack using two queues. Analyze the running time of the stack operations.


- Maintain two queues so that
    - One queue consists of the single element, the "top"
    - The other queue stores the rest of the elements.
- What does "push(x)" do:
    - enqueue "x" to the queue containing "top"
    - deqnqueue the the queue containing top. Call this element y.
    - enqueue "y" to the queue that does not contain "top".
    - O(1)
- What does pop() do:
    - move all but one element from the non-"top" queue to the "top" queue using denqueue and enqueue, but leave one element
    - denqueue the "top" queue
        - This is the return element
    - the queue that has the single element becomes the new "top" queue
    - O(n)
    